In [3]:
from pyspark.sql import SparkSession, SQLContext
import os
import socket

# create a spark session
spark_cluster_url = f"spark://{os.environ['SPARK_CLUSTER']}:7077"
spark = SparkSession.builder.master(spark_cluster_url).getOrCreate()

# test your spark connection
spark.range(5, numPartitions=5).rdd.map(lambda x: socket.gethostname()).distinct().collect()

['spark-cluster-kube-3aadmin-w-cql6x']

In [45]:
# Create a configuration for a S3 connection towards Ceph RGW
s3_region = 'us-east-1' # fill in for AWS, blank for Ceph
s3_endpoint_url = 'http://172.30.106.146:9000'
s3_access_key_id = 'minio'
s3_secret_access_key = 'minio123'
s3_bucket = 'songs'

keys = []

import boto3

# configure boto S3 connection
s3 = boto3.client('s3',
                  s3_region,
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)

for key in s3.list_objects_v2(Bucket=s3_bucket, Prefix='songs.json')['Contents']:
    keys.append('s3a://' + s3_bucket + '/' + key['Key'])
    
print(len(keys))

38


In [ ]:
# Use the above S3 values in order ro run query with Spark using the s3a connector 
hadoopConf = spark.sparkContext._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.endpoint", s3_endpoint_url)
hadoopConf.set("fs.s3a.access.key", s3_access_key_id)
hadoopConf.set("fs.s3a.secret.key", s3_secret_access_key)
hadoopConf.set("fs.s3a.path.style.access", "true")
hadoopConf.set("fs.s3a.connection.ssl.enabled", "false") # false if not https

# Print CSV values coming from the S3 bucket 
data = spark.read.json(keys)
df = data.toPandas()

In [47]:

df.head(100)

,count,songName
0,100,The Good The Bad And The Ugly
1,101,The Good The Bad And The Ugly
2,102,The Good The Bad And The Ugly
3,103,The Good The Bad And The Ugly
4,104,The Good The Bad And The Ugly
5,105,The Good The Bad And The Ugly
6,106,The Good The Bad And The Ugly
7,107,The Good The Bad And The Ugly
8,108,The Good The Bad And The Ugly
9,109,The Good The Bad And The Ugly
